<a href="https://colab.research.google.com/github/Hagar-Usama/Chest-X-ray-diagnosis/blob/improved-models/Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split
import cv2
from google.colab import drive
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
import torch.nn.functional as F
import torch
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
from sklearn.model_selection import train_test_split
from google.colab import files
from torchvision.models import resnet18, resnet50
from statistics import mode

# Load Pickle File

In [ ]:
## Credit
# https://medium.com/better-programming/load-fast-load-big-with-compressed-pickles-5f311584507e
import pickle
import bz2
import os
import cv2
import _pickle as cPickle
# Load any compressed pickle file
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = cPickle.load(data)
    return data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pathlib import Path
def ds_join():
  dataset = Path("/content/drive/My Drive/covid-19")
  p1 = dataset.joinpath("covid-compressed-label.pbz2") 
  p2 = dataset.joinpath("covid-compressed-image.pbz2") 
  return p1,p2

In [ ]:
## run iff a pickle file is used
p1, p2 = ds_join()
labels, data = decompress_pickle(p1), decompress_pickle(p2)

In [ ]:
data2 = [i.reshape((3,256,256)) for i in data]

# Optimize Data

## numerical labels

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(labels)
print(labels)

[2 2 2 ... 1 1 1]


## Data Augmentation

In [ ]:
from torchvision import transforms, utils

train_trans =  transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

valid_trans = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

# Split Data

In [ ]:
#Credit
# https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test=\
train_test_split(data2, labels, test_size=0.4, random_state=1, stratify=labels)
x_train, x_val, y_train, y_val =\
train_test_split(x_train, y_train, test_size=0.5, random_state=1, stratify=y_train) # 0.25 x 0.8 = 0.2

# Torch

In [ ]:
import torch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [ ]:
from torch.utils.data.dataset import Dataset
class MyDataset(Dataset):

    def __init__(self, data, labels, transform = None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image = self.data[index]
        # transformation
        # if self.transform:
        #   image = self.transform(image)
        #   return image.float().cuda(), torch.tensor(int(self.labels[index]))
        return torch.tensor(image).float(), torch.tensor(int(self.labels[index]))
        

## Dataloader

In [ ]:
batch_size = 32

In [ ]:
def prepare_dataloader(x_train, y_train, transform=None):
  customDataset = MyDataset(x_train, y_train, transform=transform)
  dataloader = DataLoader(customDataset, batch_size=batch_size, shuffle=True, num_workers=0)
  return dataloader

In [ ]:
# num_workers = 1 causes a crash
from torch.utils.data import DataLoader, random_split
train_dataloader = prepare_dataloader(x_train, y_train, transform= transform)
val_dataloader = prepare_dataloader(x_val, y_val, transform= transform)
test_dataloader = prepare_dataloader(x_test, y_test)

## Training  & Testing

In [ ]:
def run_model(model, dataloader, optimizer, scheduler, train = True):
  if train:
    model.train()
  else:
    model.eval()

  pred = []
  labels = []
  loss = nn.CrossEntropyLoss()
  total_loss = 0

  for (data, label) in dataloader: 
      # zero the parameter gradients
      optimizer.zero_grad()
      # forward + backward + optimize
      with torch.set_grad_enabled(train == True):
        output = model(data)
        loss_ = loss(output, label)

      total_loss += loss_.item()

      if train:
        loss_.backward()
        optimizer.step()

      pred += output.tolist()
      labels += label.tolist()
        

  return labels,pred,total_loss/len(dataloader)

In [ ]:
def train_model(model, optimizer, scheduler, epoch):

  for e in range(epoch):
    print("\n**********epoch: ", e, "\n")
    labels, pred, loss = run_model(model, train_dataloader, optimizer, scheduler, train=True)
    print("\ntrain:\n") 
    print("loss: ", loss)
    get_metrics(pred, labels)

    with torch.no_grad():
      labels, pred, loss = run_model(model, val_dataloader, optimizer, scheduler, train=False)
      scheduler.step(loss)
      print("\nvalidation:\n")
      print("loss: ", loss)
      get_metrics(pred, labels)

In [ ]:
def test_model(model, optimizer, scheduler):
  with torch.no_grad():
    labels, pred, loss = run_model(model, test_dataloader, optimizer, scheduler, train=False)
    print("test: ")
    print("loss: ", loss)
    print(pred)
    print(torch.exp(torch.tensor(pred)))
    print(labels)

    get_metrics(pred, labels)

Improved Models

In [ ]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [ ]:
def get_metrics(pred, labels):
  # pred = torch.exp(torch.tensor(pred)) # get probabilities back
  pred = torch.tensor(pred)
  _, pred = torch.max(pred, 1)
  # _, pred = torch.max(torch.tensor(pred), 1)
  # print(pred)
  f1 = f1_score(labels, pred.cpu(), average='weighted')
  r = recall_score(labels, pred.cpu(), average='weighted')
  p = precision_score(labels, pred.cpu(), average='weighted')
  a = accuracy_score(labels, pred.cpu())
  print("f1 score: ", f1)
  print("recall: ", r)
  print("precision: ", p)
  print("accuracy: ", a)

  pred_list.append(f1)
  pred_list.append(r)
  pred_list.append(p)
  pred_list.append(a)
  return f1,r,p,a

In [ ]:
dataloaders = {'train': train_dataloader, 'val':val_dataloader, 'test':test_dataloader }
dataset_sizes = {'train': len(y_train), 'val':len(y_val)}

In [ ]:
import torch.nn as nn
from torchvision import models

class MyResNet(nn.Module):
    def __init__(self, type=18, pretrained=True):
        super().__init__()
        if type == 18:
          resnet = models.resnet18(pretrained=pretrained)
        else:
          resnet = models.resnet50(pretrained=pretrained)


        # here we get all the modules(layers) before the fc layer at the end
        # note that currently at pytorch 1.0 the named_children() is not supported
        # and using that instead of children() will fail with an error
        self.features = nn.ModuleList(resnet.children())[:-1]
        # Now we have our layers up to the fc layer, but we are not finished yet 
        # we need to feed these to nn.Sequential() as well, this is needed because,
        # nn.ModuleList doesnt implement forward() 
        # so you cant do sth like self.features(images). Therefore we use 
        # nn.Sequential and since sequential doesnt accept lists, we 
        # unpack all the items and send them like this
        self.features = nn.Sequential(*self.features)
        # now lets add our new layers 
        in_features = resnet.fc.in_features
        # from now, you can add any kind of layers in any quantity!  
        # Here I'm creating two new layers 

        
        self.fc0 = nn.Linear(in_features, 256)
        self.fc0_bn = nn.BatchNorm1d(256, eps = 1e-2)
        self.fc1 = nn.Linear(256, 3)
        self.fc1_bn = nn.BatchNorm1d(3, eps = 1e-2)
        
        # initialize all fc layers to xavier
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_normal_(m.weight, gain = 1)

    def forward(self, input_imgs):
       output = input_imgs.permute(0, 3, 1, 2)
       # now in forward pass, you have the full control, 
       # we can use the feature part from our pretrained model  like this
       output = self.features(input_imgs)
       # since we are using fc layers from now on, we need to flatten the output.
       # we used the avgpooling but we still need to flatten from the shape (batch, 1,1, features)
       # to (batch, features) so we reshape like this. input_imgs.size(0) gives the batchsize, and 
       # we use -1 for inferring the rest
       output = output.view(input_imgs.size(0), -1)
       # and also our new layers.
       output = output.view(input_imgs.size(0), -1)
       # and also our new layers. 
       output = self.fc0_bn(F.relu(self.fc0(output)))
       output = self.fc1_bn(F.relu(self.fc1(output)))
       return output

# ResNet


## ResNet 18 (`pretrained = False`)

In [ ]:
import torchvision.models as models
import torch.optim as optim

model =  models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)

In [ ]:
# params_to_update = []
# for name,param in model.named_parameters():
#     if param.requires_grad == True:
#         params_to_update.append(param)
optimizer = optim.Adam(model.parameters(), 0.001, weight_decay=0.0)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.3, threshold=0.0001) 
train_model(model, optimizer, scheduler, 10)


**********epoch:  0 


train:

loss:  0.6744392298989825
f1 score:  0.7148865920586726
recall:  0.7168989547038328
precision:  0.7149108303008852
accuracy:  0.7168989547038328

validation:

loss:  0.8544118586513731
f1 score:  0.6660018695829163
recall:  0.6788511749347258
precision:  0.7318733895401915
accuracy:  0.6788511749347258

**********epoch:  1 


train:

loss:  0.4042872641649511
f1 score:  0.8546700983846105
recall:  0.8545296167247387
precision:  0.8548380235584315
accuracy:  0.8545296167247387

validation:

loss:  0.32933641307883793
f1 score:  0.887030389567136
recall:  0.886858137510879
precision:  0.8902216785826802
accuracy:  0.886858137510879

**********epoch:  2 


train:

loss:  0.30332744535472655
f1 score:  0.8910791677253919
recall:  0.8911149825783972
precision:  0.8910492465065267
accuracy:  0.8911149825783972

validation:

loss:  0.3779916597737206
f1 score:  0.8479273037530538
recall:  0.8494342906875544
precision:  0.8501649536470776
accuracy:  0.8494342906

In [ ]:
torch.save(model.state_dict(), "resNet18_false")

In [ ]:
test_model(model, optimizer, scheduler)

test: 
loss:  0.29929245604823035
[[-2.634265422821045, 2.8314340114593506, -0.8146829605102539], [1.4362465143203735, -1.5514962673187256, 0.763937771320343], [1.058443307876587, 0.8184515237808228, -2.554236650466919], [-3.372934103012085, -0.32429733872413635, 3.640029191970825], [-2.6492421627044678, 1.831264853477478, 0.31424200534820557], [-1.4990198612213135, -1.7522914409637451, 2.350494146347046], [-3.991243600845337, 3.2852554321289062, 0.42534005641937256], [-3.536245346069336, 2.729487657546997, 0.3248726427555084], [-2.835556983947754, 3.4191322326660156, -0.8271918892860413], [0.809043824672699, 0.6804847717285156, -2.33327317237854], [-0.5725676417350769, -2.644692897796631, 3.2674453258514404], [1.6080881357192993, 0.14893075823783875, -2.4820423126220703], [-2.819262742996216, 1.915103793144226, 0.4158368706703186], [-4.036880970001221, 4.150705814361572, -0.5326766967773438], [-4.00221061706543, 4.380030632019043, -0.8568487763404846], [6.330530166625977, -4.260252952

## ResNet 18 (pretrained = True)

In [ ]:
import torchvision.models as models
import torch.optim as optim

model =  models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
# params_to_update = []
# for name,param in model.named_parameters():
#     if param.requires_grad == True:
#         params_to_update.append(param)
optimizer = optim.Adam(model.parameters(), 0.001, weight_decay=0.0)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.3, threshold=0.0001) 
train_model(model, optimizer, scheduler, 10)


**********epoch:  0 


train:

loss:  0.4765957784321573
f1 score:  0.8500195124577951
recall:  0.8501742160278746
precision:  0.8501053612684673
accuracy:  0.8501742160278746

validation:

loss:  4.9148114985889855
f1 score:  0.5111029629231506
recall:  0.608355091383812
precision:  0.7707115069638123
accuracy:  0.608355091383812

**********epoch:  1 


train:

loss:  0.2316454489611917
f1 score:  0.9204187439095004
recall:  0.9207317073170732
precision:  0.9209770271090256
accuracy:  0.9207317073170732

validation:

loss:  0.2121070468177398
f1 score:  0.9255313563692812
recall:  0.9251523063533508
precision:  0.927820400176832
accuracy:  0.9251523063533508

**********epoch:  2 


train:

loss:  0.14111952343955636
f1 score:  0.9433809607618748
recall:  0.9433797909407665
precision:  0.9433884626308026
accuracy:  0.9433797909407665

validation:

loss:  0.25852689491067493
f1 score:  0.9083379328569967
recall:  0.9086161879895561
precision:  0.913536233985455
accuracy:  0.90861618798

In [ ]:
torch.save(model.state_dict(), "resNet18_true")


In [ ]:
test_model(model, optimizer, scheduler)

test: 
loss:  0.24843634835754833
[[-4.778416633605957, -2.5089728832244873, 6.358242988586426], [-7.071140766143799, -5.106512069702148, 10.697550773620605], [15.057455062866211, -11.288637161254883, -6.870029449462891], [7.536367893218994, -5.052290916442871, -3.9550490379333496], [6.712723255157471, -4.998384475708008, -2.788243532180786], [-6.634549617767334, -3.655519485473633, 9.625029563903809], [-5.299911022186279, -1.8649884462356567, 6.143850326538086], [-3.7956418991088867, 0.6897167563438416, 2.5085580348968506], [-2.100250244140625, 2.700857162475586, -0.8825761675834656], [9.011983871459961, -6.856540203094482, -3.8826396465301514], [-5.923038005828857, -5.099541664123535, 9.636500358581543], [-1.579984426498413, 2.116593360900879, -1.0120785236358643], [-6.504272937774658, -3.0328118801116943, 8.460382461547852], [-8.466806411743164, -6.26791524887085, 12.760157585144043], [-7.088183879852295, -4.588127613067627, 10.47635269165039], [-3.512254238128662, 5.478050708770752

## ResNet 50 (pretrained = False)

In [ ]:
import torchvision.models as models
import torch.optim as optim

model =  models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)

In [ ]:
optimizer = optim.Adam(model.parameters(), 0.001, weight_decay=0.0)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.3, threshold=0.0001) 
train_model(model, optimizer, scheduler, 10)


**********epoch:  0 


train:

loss:  0.9858467777570089
f1 score:  0.669783882026133
recall:  0.6672473867595818
precision:  0.6743196953833143
accuracy:  0.6672473867595818

validation:

loss:  1.2081951167848375
f1 score:  0.5040423377582797
recall:  0.5813751087902524
precision:  0.5476723628571918
accuracy:  0.5813751087902524

**********epoch:  1 


train:

loss:  0.38089604675769806
f1 score:  0.8476434017308091
recall:  0.8475609756097561
precision:  0.8477620040824655
accuracy:  0.8475609756097561

validation:

loss:  0.3637850185235341
f1 score:  0.8745724906578942
recall:  0.8746736292428199
precision:  0.878928236102793
accuracy:  0.8746736292428199

**********epoch:  2 


train:

loss:  0.28001639743645984
f1 score:  0.8945558740105816
recall:  0.8945993031358885
precision:  0.8945948059434848
accuracy:  0.8945993031358885

validation:

loss:  0.3264578464958403
f1 score:  0.8644647980498507
recall:  0.8651000870322019
precision:  0.8711893556769933
accuracy:  0.865100087

In [ ]:
torch.save(model.state_dict(), "resNet50_false")

In [ ]:
test_model(model, optimizer, scheduler)

test: 
loss:  0.40245343123873073
[[-3.348890542984009, -2.560533285140991, 4.933432102203369], [-2.6118862628936768, 2.580530881881714, -0.1038147583603859], [-4.549796104431152, 6.0195488929748535, -1.4801764488220215], [-4.879413604736328, 3.746201515197754, 0.7864037752151489], [-5.89736270904541, 6.4121270179748535, -0.8401588797569275], [-4.4812092781066895, 6.386189937591553, -1.9814174175262451], [-5.192812442779541, 0.8101729154586792, 3.5679385662078857], [3.9849538803100586, -5.125533580780029, 1.6575902700424194], [-3.018529176712036, -2.6178395748138428, 4.897507667541504], [4.233973026275635, -0.3042791187763214, -2.936371326446533], [2.249891519546509, -3.4715545177459717, 1.4588303565979004], [-5.055635929107666, 6.4950337409973145, -1.600292682647705], [-2.9193177223205566, -0.853234052658081, 3.070460081100464], [3.664374351501465, -1.3548462390899658, -1.810356855392456], [-3.5858170986175537, -2.5572736263275146, 5.224151134490967], [-5.711831092834473, 7.9027791023

## ResNet 50 (pretrained = True)

In [ ]:
import torchvision.models as models
import torch.optim as optim

model =  models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)

In [ ]:
optimizer = optim.Adam(model.parameters(), 0.001, weight_decay=0.0)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.3, threshold=0.0001) 
train_model(model, optimizer, scheduler, 10)


**********epoch:  0 


train:

loss:  0.5256167277693748
f1 score:  0.8051609942728352
recall:  0.8057491289198606
precision:  0.8068001443242853
accuracy:  0.8057491289198606

validation:

loss:  8.92648548550076
f1 score:  0.39193550479668987
recall:  0.4691035683202785
precision:  0.6930360450211761
accuracy:  0.4691035683202785

**********epoch:  1 


train:

loss:  0.23866118304431438
f1 score:  0.9110488694662545
recall:  0.9111498257839721
precision:  0.9111124154961221
accuracy:  0.9111498257839721

validation:

loss:  0.8112704306840897
f1 score:  0.8472586510237163
recall:  0.8468233246301131
precision:  0.8520351887122494
accuracy:  0.8468233246301131

**********epoch:  2 


train:

loss:  0.23206316638323995
f1 score:  0.9216549601900745
recall:  0.921602787456446
precision:  0.9223074810518389
accuracy:  0.921602787456446

validation:

loss:  0.3509993039899402
f1 score:  0.8664027390650931
recall:  0.8668407310704961
precision:  0.8769079158080044
accuracy:  0.8668407310

In [ ]:
test_model(model, optimizer, scheduler)

test: 
loss:  0.2638092488050461
[[1.8551685810089111, -7.399412155151367, 6.444147109985352], [-2.874917507171631, 5.845515251159668, -2.534027099609375], [0.22379788756370544, -2.4568536281585693, 2.900705575942993], [-2.432879686355591, -0.5562393069267273, 4.029762268066406], [-2.0866341590881348, -2.5289995670318604, 5.508114814758301], [-1.7565011978149414, 5.770102500915527, -3.5064845085144043], [4.306826114654541, -4.252654552459717, 0.47429710626602173], [5.2127604484558105, -2.9291021823883057, -1.7417105436325073], [6.281837463378906, -6.2603888511657715, 0.5113221406936646], [1.3105583190917969, 1.164428949356079, -1.95695960521698], [5.969902515411377, -6.152327060699463, 0.594708263874054], [-3.3227531909942627, 3.7473855018615723, 0.055834412574768066], [5.5713324546813965, -1.8785046339035034, -3.221323013305664], [0.2320786714553833, 0.40463268756866455, -0.20528317987918854], [-3.219498872756958, 1.6574206352233887, 2.0830466747283936], [-1.996593952178955, -4.170979

# DenseNet

## DenseNet (pretrained = False)

In [ ]:
pred_list = []

In [ ]:
from torchvision import models
model = models.densenet169(pretrained=False)
num_ftrs = model.classifier.in_features
model.fc = nn.Linear(num_ftrs, 3)
#model.cuda()
optimizer = optim.Adam(model.parameters(), 0.0001, weight_decay=0.0)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.3, threshold=0.0001) 
train_model(model, optimizer, scheduler, 10)


**********epoch:  0 


train:

loss:  3.301233539978663
f1 score:  0.7136656656979625
recall:  0.686411149825784
precision:  0.7684838521529239
accuracy:  0.686411149825784


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



validation:

loss:  1.094557782014211
f1 score:  0.8264294714760256
recall:  0.8302872062663186
precision:  0.8357829548173578
accuracy:  0.8302872062663186

**********epoch:  1 


train:

loss:  0.5179296187642548
f1 score:  0.8820149520757713
recall:  0.882404181184669
precision:  0.8825632931729079
accuracy:  0.882404181184669

validation:

loss:  0.5044812121325069
f1 score:  0.8356567863478721
recall:  0.8407310704960835
precision:  0.8592138296253613
accuracy:  0.8407310704960835

**********epoch:  2 


train:

loss:  0.3135855733934376
f1 score:  0.9013319988852104
recall:  0.9015679442508711
precision:  0.9013108041787077
accuracy:  0.9015679442508711

validation:

loss:  0.3106939688117968
f1 score:  0.8999284014231629
recall:  0.9007832898172323
precision:  0.9025375424127398
accuracy:  0.9007832898172323

**********epoch:  3 


train:

loss:  0.22629529341227478
f1 score:  0.9249168049690769
recall:  0.9250871080139372
precision:  0.925008232709453
accuracy:  0.925087108013

In [ ]:
torch.save(model.state_dict(), "denseNet_false")


In [ ]:
pred_list = []

In [ ]:
test_model(model, optimizer, scheduler)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
pred_list

[0.9199260620604759,
 0.9203655352480418,
 0.9218970362009204,
 0.9203655352480418]

## DenseNet (pretrained = True)

In [ ]:
pred_list = []

In [ ]:
import torchvision.models as models
import torch.optim as optim

model =  models.densenet169(pretrained=True)

# for param in model.parameters():
#     param.requires_grad = False

num_ftrs = model.classifier.in_features
model.fc0 = nn.Linear(num_ftrs, 600)
model.fc1 = nn.Linear(600, 200)
model.fc3 = nn.Linear(200, 3)



Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.cache/torch/hub/checkpoints/densenet169-b2777c0a.pth


In [ ]:
optimizer = optim.Adam(model.parameters(), 0.001, weight_decay=0.0)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.3, threshold=0.0001) 
train_model(model, optimizer, scheduler, 10)


**********epoch:  0 


train:

loss:  0.8679785046519505
f1 score:  0.8568689879231873
recall:  0.8371080139372822
precision:  0.8776091122643478
accuracy:  0.8371080139372822


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



validation:

loss:  14.230725526809692
f1 score:  0.5449227807520357
recall:  0.4908616187989556
precision:  0.724419251257908
accuracy:  0.4908616187989556

**********epoch:  1 


train:

loss:  0.28065795606623095
f1 score:  0.9119344010618265
recall:  0.912020905923345
precision:  0.9119320814834813
accuracy:  0.912020905923345

validation:

loss:  0.2602950479421351
f1 score:  0.9197486586205634
recall:  0.9190600522193212
precision:  0.9246646466138037
accuracy:  0.9190600522193212

**********epoch:  2 


train:

loss:  0.1856351021108114
f1 score:  0.935407948696892
recall:  0.9355400696864111
precision:  0.9354530169440562
accuracy:  0.9355400696864111

validation:

loss:  1.4389670507775412
f1 score:  0.6803040600159278
recall:  0.6962576153176675
precision:  0.8066150732508021
accuracy:  0.6962576153176675

**********epoch:  3 


train:

loss:  0.14277000642485088
f1 score:  0.9442830313033298
recall:  0.9442508710801394
precision:  0.9443351141970665
accuracy:  0.94425087108

In [ ]:
torch.save(model.state_dict(), "denseNet_true")

In [ ]:
test_model(model, optimizer, scheduler)

test: 
loss:  0.19328186152173052


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
pred_list

[0.8568689879231873,
 0.8371080139372822,
 0.8776091122643478,
 0.8371080139372822,
 0.5449227807520357,
 0.4908616187989556,
 0.724419251257908,
 0.4908616187989556,
 0.9119344010618265,
 0.912020905923345,
 0.9119320814834813,
 0.912020905923345,
 0.9197486586205634,
 0.9190600522193212,
 0.9246646466138037,
 0.9190600522193212,
 0.935407948696892,
 0.9355400696864111,
 0.9354530169440562,
 0.9355400696864111,
 0.6803040600159278,
 0.6962576153176675,
 0.8066150732508021,
 0.6962576153176675,
 0.9442830313033298,
 0.9442508710801394,
 0.9443351141970665,
 0.9442508710801394,
 0.9263453334743602,
 0.927763272410792,
 0.9319174185489969,
 0.927763272410792,
 0.9721331675853154,
 0.9721254355400697,
 0.9721563784432955,
 0.9721254355400697,
 0.9422233685846634,
 0.9425587467362925,
 0.9424844089884428,
 0.9425587467362925,
 0.9572508477799736,
 0.9573170731707317,
 0.9573128414470411,
 0.9573170731707317,
 0.8875669125253304,
 0.8885987815491732,
 0.9080276603656575,
 0.8885987815491732

# Inception v3

## preparing data

In [ ]:
torch.save(model.state_dict(), "Inception_true_auxlogit_false")

In [ ]:
from torch.utils.data import DataLoader, random_split
batch_size = 64
# resize to 299
resized = [cv2.resize(i, (299,299), interpolation = cv2.INTER_AREA) for i in data]
resized2 = [i.reshape((3,299,299)) for i in resized]

x_train, x_test, y_train, y_test=\
train_test_split(resized2, labels, test_size=0.4, random_state=1, stratify=labels)
x_train, x_val, y_train, y_val =\
train_test_split(x_train, y_train, test_size=0.5, random_state=1, stratify=y_train)

train_dataloader = prepare_dataloader(x_train, y_train, transform= train_trans)
val_dataloader = prepare_dataloader(x_val, y_val, transform= valid_trans)
test_dataloader = prepare_dataloader(x_test, y_test)

In [ ]:
print(resized2[0].shape)

(3, 299, 299)


## Inception V3 (`pretrained = False`)

In [ ]:
from torchvision import models
import torch.optim as optim

model= models.inception_v3(init_weights=True, pretrained=False, aux_logits=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), 0.001, weight_decay=0.0)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.3, threshold=0.0001) 
train_model(model, optimizer, scheduler, 10)



**********epoch:  0 


train:

loss:  0.6094233079089059
f1 score:  0.7317779637481335
recall:  0.7317073170731707
precision:  0.7321146817496305
accuracy:  0.7317073170731707

validation:

loss:  126.51620186699762
f1 score:  0.14090375663348265
recall:  0.3002610966057441
precision:  0.44087994336457237
accuracy:  0.3002610966057441

**********epoch:  1 



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



train:

loss:  0.4161035137044059
f1 score:  0.8425810339397484
recall:  0.8432055749128919
precision:  0.8429398671554837
accuracy:  0.8432055749128919

validation:

loss:  0.9975590970781114
f1 score:  0.7509114874863404
recall:  0.763272410791993
precision:  0.8067393062332966
accuracy:  0.763272410791993

**********epoch:  2 


train:

loss:  0.3155009729994668
f1 score:  0.884990665308637
recall:  0.8850174216027874
precision:  0.8851168684186256
accuracy:  0.8850174216027874

validation:

loss:  0.36193431582715774
f1 score:  0.8695283924297065
recall:  0.8694516971279374
precision:  0.8768609052851644
accuracy:  0.8694516971279374

**********epoch:  3 


train:

loss:  0.2173145595524046
f1 score:  0.917178859790071
recall:  0.9172473867595818
precision:  0.9173148329551221
accuracy:  0.9172473867595818

validation:

loss:  0.3548285812139511
f1 score:  0.879646027575141
recall:  0.8807658833768495
precision:  0.8900815117841232
accuracy:  0.8807658833768495

**********epoch:  

In [ ]:
test_model(model, optimizer, scheduler)

test: 
loss:  0.3182282031824191
[[-3.342123031616211, -1.4163413047790527, 3.952723503112793], [-5.8144426345825195, 5.5729756355285645, -0.7006058692932129], [-5.071965217590332, 5.758038520812988, -1.5617173910140991], [2.2766172885894775, -4.4648823738098145, 1.7740753889083862], [-4.720467567443848, 6.2835211753845215, -2.701657772064209], [-6.408641815185547, 6.576035022735596, -1.264389157295227], [-7.5031867027282715, 7.734842777252197, -1.541277289390564], [-4.641702175140381, 0.9899430274963379, 2.8817391395568848], [-0.6867212653160095, 1.572458028793335, -1.464813470840454], [4.805793285369873, -3.109292507171631, -2.070469856262207], [6.854283332824707, -2.8438775539398193, -4.412267208099365], [-7.087599754333496, 5.860260963439941, 0.1010245680809021], [7.016898155212402, -3.6529617309570312, -3.5484938621520996], [0.03538564220070839, -2.399468421936035, 1.8312358856201172], [9.35315227508545, -4.289499759674072, -5.022781848907471], [-2.470986843109131, 2.2154927253723

## Inception V3 (`pretrained = True`)

In [ ]:
model= models.inception_v3(init_weights=True, pretrained=True, aux_logits=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), 0.001, weight_decay=0.0)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.3, threshold=0.0001) 
train_model(model, optimizer, scheduler, 10)



**********epoch:  0 


train:

loss:  0.39387105736467576
f1 score:  0.8456790815172874
recall:  0.8458188153310104
precision:  0.8456155341165759
accuracy:  0.8458188153310104

validation:

loss:  1.3022183510992262
f1 score:  0.7696001333370689
recall:  0.7745865970409052
precision:  0.8077273472086212
accuracy:  0.7745865970409052

**********epoch:  1 


train:

loss:  0.18946712132957247
f1 score:  0.9336910235016472
recall:  0.9337979094076655
precision:  0.9339864538966854
accuracy:  0.9337979094076655

validation:

loss:  0.38427844478024376
f1 score:  0.8717041561087125
recall:  0.8729329852045257
precision:  0.8804288011180773
accuracy:  0.8729329852045257

**********epoch:  2 


train:

loss:  0.1174387898710039
f1 score:  0.9625788059132383
recall:  0.9625435540069687
precision:  0.9626931486123999
accuracy:  0.9625435540069687

validation:

loss:  0.4307142131858402
f1 score:  0.857865333220295
recall:  0.8633594429939078
precision:  0.8830795019978481
accuracy:  0.8633594

In [ ]:
torch.save(model.state_dict(), "Inception_true_auxlogit_false")

In [ ]:
test_model(model, optimizer, scheduler)

test: 
loss:  0.1812179209664464
[[8.096827507019043, -4.9308881759643555, -4.567342281341553], [3.1655690670013428, -1.9899612665176392, -1.8506672382354736], [-2.3525118827819824, 1.1842247247695923, 0.5925988554954529], [-8.987308502197266, -5.421750068664551, 12.349607467651367], [-4.13490104675293, 4.725791931152344, -2.011983633041382], [-5.24079704284668, 6.264619827270508, -2.887295961380005], [-1.6977537870407104, 3.714656114578247, -3.204270601272583], [-2.3615548610687256, 4.172900199890137, -3.086946725845337], [5.8595781326293945, -3.801274061203003, -3.090831756591797], [0.9617079496383667, -1.3152446746826172, -0.07232923060655594], [-7.930696964263916, -3.0846211910247803, 9.123174667358398], [-3.8922717571258545, 6.045253753662109, -3.917367696762085], [-4.0352959632873535, -1.5396720170974731, 4.676162242889404], [-1.4276559352874756, 3.488633155822754, -3.158393383026123], [8.335271835327148, -5.588136672973633, -4.212161540985107], [-7.802392959594727, -3.6415987014